In [ ]:
#IMPORT LIBRARIES
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [ ]:
#for performing accurate word and sentence tokenization in NLTK-based natural language processing projects.
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Data source : https://www.consumerfinance.gov/data-research/consumer-complaints/

In [ ]:
df=pd.read_csv('/content/consumer_complaints_test.csv')

In [ ]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID


In [ ]:
df.shape

(7985, 18)

In [ ]:
df.dtypes

Date received                    object
Product                          object
Sub-product                      object
Issue                            object
Sub-issue                        object
Consumer complaint narrative     object
Company public response          object
Company                          object
State                            object
ZIP code                         object
Tags                             object
Consumer consent provided?       object
Submitted via                    object
Date sent to company             object
Company response to consumer     object
Timely response?                 object
Consumer disputed?              float64
Complaint ID                      int64
dtype: object

In [ ]:
df.isnull().sum()

Date received                   0   
Product                         0   
Sub-product                     0   
Issue                           0   
Sub-issue                       0   
Consumer complaint narrative    6815
Company public response         0   
Company                         0   
State                           0   
ZIP code                        0   
Tags                            0   
Consumer consent provided?      1494
Submitted via                   0   
Date sent to company            0   
Company response to consumer    0   
Timely response?                0   
Consumer disputed?              7985
Complaint ID                    0   
dtype: int64

In [ ]:
df_cleaned = df.dropna(subset='Consumer complaint narrative')
df_cleaned

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
11,07/28/20,Debt collection,Credit card debt,Threatened to contact someone or share information improperly,Contacted your employer,XXXX XXXX has called me from Midwest XXXX agencies. I have spoke with him and tried to settle the debt but he wouldnt give me accurate information. The numbers just do not add up. He also contacted my previous employer as well as my current employer and my girlfriend harrassing us. I have told them repeatedly i am on the national do not call registry and he still continues to call. He told my boss information that was private and hurt my employment which eventually lead to me having to leave the company. I am sick and tired of being harrassed by this same individual. He used profane language on the phone calls and calls me at least once every other day. I have tried to settle debt with the original collector and they will not accept any form of payment if it isnt the full amount.,Company has responded to the consumer and the CFPB and chooses not to provide a public response,"Midwest Service Bureau, Inc.",AR,None,None,Consent provided,Web,07/28/20,Closed with explanation,Yes,NaN,3767821
14,08/01/20,Credit card or prepaid card,Government benefit card,Problem getting a card or closing an account,Don't want a card provided by your employer or the government,"I received in the mail a US Bank ReliaCard Debit card in the mail. Never solicited for it. I called the phone number and discovered its for unemployment insurance payment. I called XXXX to have the card deactivated. I was on hold for 2 hrs before reaching an agent XXXX located in XXXX, Florida. I advised I never filed a claim for unemployment. She deactivated the card, she advised she will mail me paperwork showing the card is deactivated. XXXX advised me to file a Police Report, file a complaint with the Federal Trade Commission, file reports of fraud with the 3 credit bureau 's and call the Colorado Unemployment and get the case closed on my name. I have a very unusual last name XXXX. I'm the only one in Colorado!! I will do some of this today and the rest I will have to do on Monday with phone calls. \nI called the XXXX County Sheriff 's Department to file a Police Report it is CT XXXX with Officer XXXX Badge # XXXX. He informed me that these people can be anywhere and to file a report with this agency as well. \nI did not activate the card, I refused to give the last 4 digits or confirm my phone number. The card was never activated. I am a Federal Worker for USPS, I know there was a breach a few years back on our information. I will be enrolling in XXXX as well for my protection.",Company has responded to the consumer and the CFPB and chooses not to provide a public response,U.S. BANCORP,CO,800XX,Servicemember,Consent provided,Web,08/01/20,Closed with explanation,Yes,NaN,3775920
16,08/10/20,Mortgage,Conventional home mortgage,Trouble during payment process,None,"In response to complaint XXXX : US Bank agreed to waive all late fees that had been applied to my account. I was assured this would be corrected by Thursday, XXXX. I logged into the payment portal tonight and I see they have applied another late fee, I can only imagine in retaliation to my complaint.",Company has responded to the consumer and the CFPB and chooses not to provide a public response,U.S. BANCORP,VA,221XX,None,Consent provided,Web,08/10/20,Closed with explanation,Yes,NaN,3786948
20,07/29/20,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,"A collection appeared on my report on XX/XX/2019, and they are demanding {$5200.00} in which i do not owe. i am not sure where this came from or what this is but it is significantly affecting my reputation. this must be inve

In [ ]:
df_cleaned['Product'].value_counts()

Debt collection                444
Credit card or prepaid card    323
Mortgage                       240
Checking or savings account    115
Vehicle loan or lease          41 
Student loan                   7  
Name: Product, dtype: int64

Least product is student loans.

Rename the complaints column

In [ ]:
complaints_df=df_cleaned[['Consumer complaint narrative']].rename(columns={'Consumer complaint narrative':'complaints'})

In [ ]:
pd.set_option('display.max_colwidth', -1)
complaints_df

<ipython-input-72-704451e25f62>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,complaints
11,XXXX XXXX has called me from Midwest XXXX agencies. I have spoke with him and tried to settle the debt but he wouldnt give me accurate information. The numbers just do not add up. He also contacted my previous employer as well as my current employer and my girlfriend harrassing us. I have told them repeatedly i am on the national do not call registry and he still continues to call. He told my boss information that was private and hurt my employment which eventually lead to me having to leave the company. I am sick and tired of being harrassed by this same individual. He used profane language on the phone calls and calls me at least once every other day. I have tried to settle debt with the original collector and they will not accept any form of payment if it isnt the full amount.
14,"I received in the mail a US Bank ReliaCard Debit card in the mail. Never solicited for it. I called the phone number and discovered its for unemployment insurance payment. I called XXXX to have the card deactivated. I was on hold for 2 hrs before reaching an agent XXXX located in XXXX, Florida. I advised I never filed a claim for unemployment. She deactivated the card, she advised she will mail me paperwork showing the card is deactivated. XXXX advised me to file a Police Report, file a complaint with the Federal Trade Commission, file reports of fraud with the 3 credit bureau 's and call the Colorado Unemployment and get the case closed on my name. I have a very unusual last name XXXX. I'm the only one in Colorado!! I will do some of this today and the rest I will have to do on Monday with phone calls. \nI called the XXXX County Sheriff 's Department to file a Police Report it is CT XXXX with Officer XXXX Badge # XXXX. He informed me that these people can be anywhere and to file a report with this agency as well. \nI did not activate the card, I refused to give the last 4 digits or confirm my phone number. The card was never activated. I am a Federal Worker for USPS, I know there was a breach a few years back on our information. I will be enrolling in XXXX as well for my protection."
16,"In response to complaint XXXX : US Bank agreed to waive all late fees that had been applied to my account. I was assured this would be corrected by Thursday, XXXX. I logged into the payment portal tonight and I see they have applied another late fee, I can only imagine in retaliation to my complaint."
20,"A collection appeared on my report on XX/XX/2019, and they are demanding {$5200.00} in which i do not owe. i am not sure where this came from or what this is but it is significantly affecting my reputation. this must be investigated and fixed. there are several other accounts involved also."
38,"Voss and Klein, on behalf of XXXX XXXX XXXX XXXX, is reporting an astronomical amount of debt on my credit report that they are constantly increasing by almost {$100.00} every couple weeks without sending me anything at all in regards to this debt. A man named XXXX reached out to me in 2018 and at that time the amount he wanted me to pay was already more than double what I owed to the hospital. I questioned the amount, asked for explanations from their company, and asked for confirmation of the specific amount the hospital was saying I owed as I was willing to work out something to pay what I actually owed, and he was incredibly rude and aggressive. He was unwilling to work with me would not accept a payment for the amount owed, and made threats about ruining my credit. I did not speak with him or anyone in the company again because I felt very uncomfortable talking to them. Since then, I've received nothing from the company in regards to this debt and they just continue to increase the amount. My last statement from the hospital was that I had owed them {$1700.00} and the collection agency is reporting that I owe almost {$10000.00}"
...,...
7962,"Your company has sent a letter to the Credit Bureaus alleging verification that I was delinquent with a collec

Import libraries

In [ ]:
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups

In [ ]:
X_train, X_hold = train_test_split(complaints_df, test_size=0.3, random_state=111)

In [ ]:
#stemmer = PorterStemmer()
stemmer = nltk.stem.SnowballStemmer('english')
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
def tokenize(text):
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2 and len(re.sub('\d+', '', word.strip('Xx/'))) > 3) ]
   tokens = map(str.lower, tokens)
   stems = [stemmer.stem(item) for item in tokens if (item not in stop_words)]
   return stems

In [ ]:


# Create the TF-IDF vectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words=None, max_df=0.75, max_features=1000, lowercase=False, ngram_range=(1, 2))

# Transform the text data
tfidf_vectors = vectorizer.fit_transform(X_train.complaints)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
tfidf_vectors.A

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.34265884,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
vectorizer.get_feature_names_out()

array(['abil', 'abl', 'absolut', 'abus', 'accept', 'access', 'accid',
       'accord', 'account', 'account accur', 'account balanc',
       'account call', 'account close', 'account collect',
       'account credit', 'account number', 'account open', 'account paid',
       'account show', 'account still', 'account would', 'accru', 'acct',
       'accur', 'acknowledg', 'act', 'action', 'activ', 'actual', 'ad',
       'addit', 'address', 'adjust', 'admit', 'advanc', 'advertis',
       'advis', 'affect', 'affect credit', 'agenc', 'agent', 'agre',
       'agreement', 'airlin', 'alert', 'alleg', 'alleg debt', 'allow',
       'almost', 'along', 'alreadi', 'alreadi paid', 'also', 'although',
       'alway', 'amazon', 'amount', 'amount owe', 'annual', 'anoth',
       'answer', 'anymor', 'anyon', 'anyth', 'anyway', 'apart', 'apolog',
       'appar', 'appear', 'appli', 'applic', 'apprais', 'appreci',
       'appropri', 'approv', 'approxim', 'around', 'arrang', 'ask',
       'assess', 'assign', '

In [ ]:
clf = decomposition.NMF(n_components=6, random_state=111)

W1 = clf.fit_transform(tfidf_vectors)
H1 = clf.components_

In [ ]:
H1

array([[0.        , 0.07211244, 0.01360992, ..., 0.0837589 , 0.00749157,
        0.07022799],
       [0.01046213, 0.        , 0.00383772, ..., 0.00446648, 0.00626606,
        0.06773149],
       [0.00640892, 0.00549733, 0.01234641, ..., 0.02821527, 0.0136962 ,
        0.07812927],
       [0.00290896, 0.06907849, 0.01414857, ..., 0.00753123, 0.        ,
        0.00914277],
       [0.        , 0.011959  , 0.        , ..., 0.02066143, 0.01860803,
        0.00110525],
       [0.03093523, 0.10168083, 0.        , ..., 0.03334412, 0.01580321,
        0.13901327]])

In [ ]:
W1

array([[0.10197549, 0.        , 0.02535107, 0.        , 0.        ,
        0.00114816],
       [0.        , 0.16528174, 0.        , 0.        , 0.        ,
        0.        ],
       [0.02642768, 0.1741803 , 0.03094966, 0.0162681 , 0.01986342,
        0.        ],
       ...,
       [0.        , 0.20685625, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00451217, 0.01223597, 0.00193248,
        0.09836285],
       [0.        , 0.        , 0.        , 0.05835845, 0.00215599,
        0.09480824]])

In [ ]:

# let us select a random of 10 words
num_words=15

vocab = np.array(vectorizer.get_feature_names_out())
#Sort the words based on the arugument sort
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]
#displays the topic names with highest coeffients

In [ ]:
topics

['call told bank would account receiv check said phone back number ask servic time email',
 'debt report collect credit credit report compani disput account valid remov verifi agenc alleg inform document',
 'card credit credit card charg close account limit citi bank citibank credit limit score synchroni balanc open',
 'payment late month made made payment late payment credit statement account report fee make late fee balanc make payment',
 'well fargo fargo well account check deposit bank check account open bonus envelop offer author money mail',
 'loan mortgag home freedom compani lender freedom mortgag escrow servic insur document refin mortgag compani rate email']

In [ ]:
colnames = ["Topic" + str(i) for i in range(clf.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [ ]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.10,0.00,0.03,0.00,0.00,0.00,0
Doc1,0.00,0.17,0.00,0.00,0.00,0.00,1
Doc2,0.03,0.17,0.03,0.02,0.02,0.00,1
Doc3,0.05,0.02,0.01,0.02,0.01,0.03,0
Doc4,0.00,0.00,0.26,0.03,0.00,0.00,2
...,...,...,...,...,...,...,...
Doc814,0.08,0.00,0.16,0.05,0.04,0.00,2
Doc815,0.07,0.00,0.00,0.03,0.00,0.03,0
Doc816,0.00,0.21,0.00,0.00,0.00,0.00,1
Doc817,0.00,0.00,0.00,0.01,0.00,0.10,5


In [ ]:

WHold = clf.transform(vectorizer.transform(X_hold.complaints[:5]))

In [ ]:
colnames = ["Topic" + str(i) for i in range(clf.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [ ]:
df_doc_topic['dominant_topic']

Doc0      0
Doc1      1
Doc2      1
Doc3      0
Doc4      2
         ..
Doc814    2
Doc815    0
Doc816    1
Doc817    5
Doc818    5
Name: dominant_topic, Length: 819, dtype: int64

In [ ]:

X_hold.head()

,complaints
6147,"I have a Synchrony Lowes credit cards. I paid the account in full. There was an interest charge that was added to the account for approximately {$4.00} that became overdue. I never received a bill, statement, or phone call to let me know that this amount was past due. This resulted in Lowes reporting negative payment information on my credit report."
433,"On XX/XX/2020 I closed my Visa Credit Card Account. I sent a final payment which bank never received. Now my credit report with XXXX is reflecting and increasing the amount of deficit I owe. I called Synchrony Bank/Gap Visa and made a final payment again, registered mail. The problem here is that there are at least 5 different mailing addresses for this credit payment. I forwarded all this info to XXXX, filled out their dispute form and would like to include XXXX in this complaint as XXXX and XXXX had no problem handling all of this credit XXXX over the phone and resolved it. \nI believe Synchrony Bank/Gap Visa has a major problem with keeping track of credit card payments and handling its customers thru XXXX. XXXX seem to relish all the paperwork and hoops that they make their XXXX-rated consumers have to jump thru to get a simple job done."
1778,"Listen, i've tried several times to get this account resolved but unfortunately, i haven't been successful as of yet. I've sent all the documetns and forms that were requested of affidavit and it still reports on my credit. The only thing i got back from National Recovery COllection company are bunch of pre-printed statement. Again, I have disputed this item with the credit reporting agency and they reported you confirmed the account as valid. I honestly do not believe to ever have any relationship with this collection agency or original creditor. Also there are several inaccuracies with this account the way it's reporting and that's a total violation of FCRA and FDCPA laws and regulations. In a good faith effort to resolve the matter amicably, I must demand proof of this debt, specifically the alleged contract or other instrument bearing my signature, as well as proof of your authority in this matter. Absent such proof, you must correct any erroneous reports of this past debt as mine. I am writing to request that you please provide the following information : 1. Please evidence your authorization under 15 USC 1692 ( e ) and 15 USC 1692 ( f ) in this alleged matter. 2. What is your authorization of law for your collection of information? 3. What is your authorization of law for your collection of this alleged debt? 4. Please evidence your authorization to do business or operate in this state. 5. Please evidence proof of the alleged debt, including the alleged contract or other instrument bearing my signature. 6. Please provide a complete account history, including any charges added for collection activity."
457,Collection company keeps putting erroneous and misleading information on my credit report. They keep taking it off and replacing it back on. I believe it to be criminal at this point and I am willing to travel anywhere for a deposition or court proceedings.
6368,On XX/XX/2020 I made an online purchase for {$17.00} from XXXX XXXX XXXX. The telephone number is XXXX. When I did not receive the item about a month later I called and the phone number is disconnected. I can not find the company. \n\nMy complaint is against US Bank XXXX. I called to dispute the charge and originally they took it off while they investigated. This was in XXXX. Then on XX/XX/2020 they re-charged it and I got a letter saying I agreed to the charge. THAT IS FALSE - I NEVER AGREED TO IT BECAUSE I NEVER GOT THE MERCHANDISE AND THE COMPANY DOESN'T APPEAR TO EXIST. \n\nI also had another charge that was fraudulent and apparently they determined that this wasn't fraud because I ordered it. BUT I said I wanted to dispute it. \n\nI have called numerous times and I keep being told that is it being investigated and a supervisor will call me back

In [ ]:
# Calculate reconstruction error (Frobenius norm)
reconstructed_data = np.dot(W1, H1)
#reconstruction_error = np.sqrt(mean_squared_error(reconstructed_data))

# Print reconstruction error
print("Reconstruction Error:", reconstructed_data )


Reconstruction Error: [[0.00019799 0.00760981 0.00170087 ... 0.00929493 0.00112931 0.0093018 ]
 [0.0017292  0.         0.00063431 ... 0.00073823 0.00103567 0.01119478]
 [0.00206797 0.00343723 0.00164042 ... 0.00439771 0.00208292 0.01624222]
 ...
 [0.00216416 0.         0.00079386 ... 0.00092392 0.00129617 0.01401068]
 [0.00310739 0.01089477 0.00022883 ... 0.00353921 0.00165221 0.01414028]
 [0.00310268 0.01369728 0.00082569 ... 0.00364535 0.00153839 0.01371554]]


Reconstruction Error:
Reconstruction error measures how well the original data can be reconstructed from the factorized matrices produced by NMF. A lower reconstruction error indicates a better fit. This metric is particularly useful for dimensionality reduction tasks.

Topic Coherence:
Measures the semantic coherence of the words within each topic. Higher coherence values indicate more meaningful and coherent topics.

In [ ]:
#Here low value of topic coherence is reported.